In [1]:
import Random

In [2]:
include("Storage.jl")
include("GillespieFunctions.jl")
include("Rates.jl")

tt_protein_fb_rates! (generic function with 1 method)

In [5]:
function balance_test(initialstate::Vector{Int64},params::Vector{Float64},numsteps::Int64,maxsteps::Int64,rng::Random.AbstractRNG)
    statevector = initialstate
    outcomes = [1 0 ; -1 0; 0 1 ; 0 -1]
    calcstor = BalanceEqData()

    endstate,steps = direct_gillespie!(initialstate,params,outcomes,tt_protein_fb_rates!,calcstor,updatestorage!,rng,numsteps,maxsteps,10000)

    println("Flux Balance on x")
    println(relative_error(calcstor.meanRpX,calcstor.meanRmX))

    println("Flux Balance on y")
    println(relative_error(calcstor.meanRpY,calcstor.meanRmY))

    println("Cov Balance on x,x")
    println(cov_balance_comp(calcstor.CXRmX/calcstor.weightsum,calcstor.CXRmX/calcstor.weightsum,
                            calcstor.meanRpX+calcstor.meanRmX,calcstor.CXRpX/calcstor.weightsum,calcstor.CXRpX/calcstor.weightsum))
    println("Cov Balance on y,y")
    println(cov_balance_comp(calcstor.CYRmY/calcstor.weightsum,calcstor.CYRmY/calcstor.weightsum,
                            calcstor.meanRpY+calcstor.meanRmY,calcstor.CYRpY/calcstor.weightsum,calcstor.CYRpY/calcstor.weightsum))
    println("Cov Balance on y,x")
    println(cov_balance_comp(calcstor.CYRmX/calcstor.weightsum,calcstor.CXRmY/calcstor.weightsum,
                            0,calcstor.CYRpX/calcstor.weightsum,calcstor.CXRpY/calcstor.weightsum))
    println(steps)
    println(endstate)
    println(calcstor.meanx)
    println(calcstor.meany)


    return calcstor
end

balance_test (generic function with 1 method)

In [6]:
@time calcs = balance_test([10,50],[10.,1.,4.,1.,-1.,50.],10^7,10^10,Random.MersenneTwister())

TypeError: TypeError: in typeassert, expected Type, got a value of type Int64

In [6]:
function reach_balance_test(initialstate,params,numsteps,maxsteps,rng,flux_thres,cov_thres,tries)
    statevector = initialstate
    outcomes = [1 0 ; -1 0; 0 1 ; 0 -1]
    calcstor = BalanceEqData()
    trynum = 0
    thresholdmet = false
    totalsteps = 0
    
    while (trynum < tries) && (!thresholdmet)
        new_max = trunc(Int,maxsteps*(tries-trynum)/tries)
        statevector,steps = direct_gillespie!(statevector,params,outcomes,tt_protein_fb_rates!,calcstor,updatestorage!,rng,numsteps,new_max)
        if (relative_error(calcstor.meanRpX,calcstor.meanRmX) < flux_thres) &&  (relative_error(calcstor.meanRpY,calcstor.meanRmY) < flux_thres)
            cbxx_comp = cov_balance_comp(calcstor.CXRmX/calcstor.weightsum,calcstor.CXRmX/calcstor.weightsum,
            calcstor.meanRpX+calcstor.meanRmX,calcstor.CXRpX/calcstor.weightsum,calcstor.CXRpX/calcstor.weightsum) < cov_thres
            cbyy_comp = cov_balance_comp(calcstor.CYRmY/calcstor.weightsum,calcstor.CYRmY/calcstor.weightsum,
                            calcstor.meanRpY+calcstor.meanRmY,calcstor.CYRpY/calcstor.weightsum,calcstor.CYRpY/calcstor.weightsum) < cov_thres
            cbyx_comp = cov_balance_comp(calcstor.CYRmX/calcstor.weightsum,calcstor.CXRmY/calcstor.weightsum,
                            0,calcstor.CYRpX/calcstor.weightsum,calcstor.CXRpY/calcstor.weightsum) <cov_thres 
            thresholdmet = all([cbxx_comp,cbyy_comp,cbyx_comp])               
        end
        trynum += 1
        totalsteps += steps
        print("Balance Test for chained simulation ")
        println(trynum)
        print("Current steps: ")
        println(totalsteps)
        print("Means: ")
        println([calcstor.meanx,calcstor.meany])
        print("Flux Balance on x: ")
        println(relative_error(calcstor.meanRpX,calcstor.meanRmX))
    
        print("Flux Balance on y: ")
        println(relative_error(calcstor.meanRpY,calcstor.meanRmY))
    
        print("Cov Balance on x,x: ")
        println(cov_balance_comp(calcstor.CXRmX/calcstor.weightsum,calcstor.CXRmX/calcstor.weightsum,
                                calcstor.meanRpX+calcstor.meanRmX,calcstor.CXRpX/calcstor.weightsum,calcstor.CXRpX/calcstor.weightsum))
        print("Cov Balance on y,y: ")
        println(cov_balance_comp(calcstor.CYRmY/calcstor.weightsum,calcstor.CYRmY/calcstor.weightsum,
                                calcstor.meanRpY+calcstor.meanRmY,calcstor.CYRpY/calcstor.weightsum,calcstor.CYRpY/calcstor.weightsum))
        print("Cov Balance on y,x: ")
        println(cov_balance_comp(calcstor.CYRmX/calcstor.weightsum,calcstor.CXRmY/calcstor.weightsum,
                                0,calcstor.CYRpX/calcstor.weightsum,calcstor.CXRpY/calcstor.weightsum))
    end
    print("Tries taken: ")
    println(trynum)
    print("Total Steps: ")
    println(totalsteps)
    print("Threshold met: ")
    println(thresholdmet)
return calcstor
end

reach_balance_test (generic function with 1 method)

In [7]:
calcs = reach_balance_test([100,400],[10.,1.,4.,1.,-1.,50.],10^5,10^8,Random.MersenneTwister(),0.01,0.03,20)

Balance Test for chained simulation 1
Current steps: 997974
Means: [6.57057141312062, 26.376926330873413]
Flux Balance on x: 0.00892289716151807
Flux Balance on y: 0.005842130438943456
Cov Balance on x,x: 0.02427694083540527
Cov Balance on y,y: 0.050038497913822484
Cov Balance on y,x: 0.13096847414610852
Balance Test for chained simulation 2
Current steps: 1998000
Means: [6.620410909557105, 26.47342223045691]
Flux Balance on x: 8.509374977335167e-5
Flux Balance on y: 0.0019331219228898122
Cov Balance on x,x: 0.009997237479179216
Cov Balance on y,y: 0.015254255302058735
Cov Balance on y,x: 0.07778851582944875
Balance Test for chained simulation 3
Current steps: 2995907
Means: [6.603983358935657, 26.384350315960365]
Flux Balance on x: 0.0034403890753661186
Flux Balance on y: 0.0010826328733251369
Cov Balance on x,x: 0.006102194832940691
Cov Balance on y,y: 0.00041909272209214056
Cov Balance on y,x: 0.05945942945101689
Balance Test for chained simulation 4
Current steps: 3995867
Means: [6

BalanceEqData(6.618567020190766, 26.422240276260876, 287566.84179239837, 1.6733083994126217e6, 1.8186760572627004e7, 2.608366918014273e6, 6.610986684710762, 6.6065245258885605, 26.426098103554242, 26.434195014520032, 1.6461330633552896e6, -214017.11355457047, 2.680975135388464e6, -1.5163946118314483e6, 2.5421886371082687e6, 6.5845322534211585e6, 1.8064923938876312e7, 1.0723900541553857e7)